Em primeira instãncia , realizamos a instalação das bibliotecas:

In [1]:
!pip install fasttext-wheel

In [2]:
import fasttext

In [3]:
!pip install pandas
!pip install numpy

In [4]:
import pandas as pd
import numpy as np

import gzip
import json

Abertura do arquivo:

In [5]:
file_path = "C:\\Caio\\Owl\\aotm2011_playlists.json.gz"
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    data = json.load(f)

Nosso DataSet das playlists em formato de DataFrame, e sem alterações prévias:

In [6]:
playlists = pd.read_json(file_path)
playlists

,category,filtered_lists,playlist,timestamp,mix_id,user
0,Mixed Genre,"[[SOFDPDC12A58A7D198], [SOPIEQP12A8C13F268, SO...","[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,89567,"{'mixes_posted': '23', 'member_since': '2004-0..."
1,Theme,"[[SOQOVCB12A58A7E83B], [SOGKLYN12AB017F155, SO...","[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,117124,"{'mixes_posted': '7', 'member_since': '2006-11..."
2,Theme,[[SOFIQBR12A6D4F78C6]],"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,97418,"{'mixes_posted': '93', 'member_since': '2003-1..."
3,Theme,"[[SORJUFZ12A67ADAF25, SOEELNN12AB018AC8D], [SO...","[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,23876,"{'mixes_posted': '8', 'member_since': '2002-03..."
4,Theme,"[[SOOUYTP12B0B8094B6, SOAJODS12A8C138A23], [SO...","[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,70694,"{'mixes_posted': '60', 'member_since': '2004-0..."
...,...,...,...,...,...,...
101338,Hip Hop,"[[SODLSVK12A8C13EEE1], [SOEOMEJ12A8AE46299], [...","[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,102561,"{'mixes_posted': '24', 'member_since': '2003-0..."
101339,Mixed Genre,"[[SONYEIJ12A6D4FA32D, SOSGUDA12A6D4FA2FB], [SO...","[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,69379,"{'mixes_posted': '1', 'member_since': '2004-03..."
101340,Single Artist,[[SOYOFST12D0219B4BA]],"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,18933,"{'mixes_posted': '23', 'member_since': '2001-1..."
101341,Theme,"[[SOKAWYU12A6D4F6769, SOXMBFV12A8C13B8DD], [SO...","[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,100251,"{'mixes_posted': '40', 'member_since': '2002-0..."


Pegaremos apenas as colunas que nos parecem interessantes:

In [7]:
playlists = playlists.drop(columns = ['user', 'mix_id', 'filtered_lists'])

In [8]:
playlists

,category,playlist,timestamp
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00
...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00


Apartir de agora, iremos testar um modelo para remover alguns caracteres indesejados da coluna playlist, como os colchetes por exemplo.

In [9]:
song_list = (playlists.iloc[1]['playlist'])
cleaned_list = [f"{band}::{song}" for (band, song), _ in song_list]
cleaned_list = ';'.join(cleaned_list)
print(cleaned_list)

The Cure::Edge Of the Deep Green Sea;Jeff Buckley::Nightmares By the Sea;Travis::Driftwood;Johnny Cash::Big River;G.Love::Fishing Song;REM::Nightswimming;The Doves::Caught By the River;Sugar::A Good Idea;Badly Drawn Boy::River, Sea, Ocean;Apples In Stereo::Stream Running Over;Frank Black & The Catholics::St. Francis Damn Disaster;The Tragically Hip::New Orleans Is Sinking;Tom Waits::Rain;Morphine::Wishing Well;They Might Be Giants::Cowtown;Talking Heads::Take Me To the River;Mike Doughty::Down On the River By the Sugar Plant;Porno For Pyros::Tahitian Moon;James::Sometimes


Removemos os espaços também

In [10]:
cleaned_list = cleaned_list.replace(' ', '')
cleaned_list

'TheCure::EdgeOftheDeepGreenSea;JeffBuckley::NightmaresBytheSea;Travis::Driftwood;JohnnyCash::BigRiver;G.Love::FishingSong;REM::Nightswimming;TheDoves::CaughtBytheRiver;Sugar::AGoodIdea;BadlyDrawnBoy::River,Sea,Ocean;ApplesInStereo::StreamRunningOver;FrankBlack&TheCatholics::St.FrancisDamnDisaster;TheTragicallyHip::NewOrleansIsSinking;TomWaits::Rain;Morphine::WishingWell;TheyMightBeGiants::Cowtown;TalkingHeads::TakeMeTotheRiver;MikeDoughty::DownOntheRiverBytheSugarPlant;PornoForPyros::TahitianMoon;James::Sometimes'

*Importamos aqui uma biblioteca para ajudar na manipulação do texto

In [11]:
import re

Fazemos então uma função que agrupa os passos anteriores: remove os colchetes, retira os espaços e faz devidas alterações na capitalização da palavra(MAIÚSCULA Ou minúscula)

In [12]:
def limpador(song_list):
    def capitalize_words(text):
        return ''.join(word.capitalize() for word in text.split())

    cleaned_list = [f"{capitalize_words(band)}::{capitalize_words(song)}" for (band, song), _ in song_list]
    cleaned_list = (';'.join(cleaned_list)).replace(" ", "")
    return cleaned_list

Criamos então uma nova coluna, com todos os termos da coluna 'playlist' e aplicamos a função anterior em todas as linhas


In [13]:
playlist_limpa = playlists.get('playlist').apply(limpador)

In [14]:
playlists = playlists.assign(
    playlists_limpas = playlist_limpa
)
playlists

,category,playlist,timestamp,playlists_limpas
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,PeterMurphy::MarleneDietrich'sFavouritePoem;Th...
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,TheCure::EdgeOfTheDeepGreenSea;JeffBuckley::Ni...
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,PeterGabriel::TheFeelingBegins;OfraHazaAndIggy...
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,TheAnimals::SanFranciscoNights;TheZombies::Tim...
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,Sparklehorse::It'sAWonderfulLife;Whiskeytown::...
...,...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,R.Kelly::BurnItUp(remixFeat.DaddyYankee);50Cen...
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,Pachelbel::Grave;Prefuse73::AfternoonLoveIn;Ni...
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,Queen::BohemianRhapsody;*::TieYourMotherDown;*...
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,BirthdayParty::Big-jesus-trash-can;XmalDeutsch...


Novamente, organizamos as colunas e filtramos apenas a que iremos usar:

In [15]:
playlists = playlists[['category','playlists_limpas', 'timestamp']]
playlists

,category,playlists_limpas,timestamp
0,Mixed Genre,PeterMurphy::MarleneDietrich'sFavouritePoem;Th...,2005-03-27 10:53:00
1,Theme,TheCure::EdgeOfTheDeepGreenSea;JeffBuckley::Ni...,2008-01-11 21:45:00
2,Theme,PeterGabriel::TheFeelingBegins;OfraHazaAndIggy...,2005-10-31 10:15:00
3,Theme,TheAnimals::SanFranciscoNights;TheZombies::Tim...,2002-03-20 17:59:00
4,Theme,Sparklehorse::It'sAWonderfulLife;Whiskeytown::...,2004-03-23 03:43:00
...,...,...,...
101338,Hip Hop,R.Kelly::BurnItUp(remixFeat.DaddyYankee);50Cen...,2006-04-03 14:54:00
101339,Mixed Genre,Pachelbel::Grave;Prefuse73::AfternoonLoveIn;Ni...,2004-03-03 15:05:00
101340,Single Artist,Queen::BohemianRhapsody;*::TieYourMotherDown;*...,2001-12-18 00:01:00
101341,Theme,BirthdayParty::Big-jesus-trash-can;XmalDeutsch...,2006-01-28 10:16:00


Aqui , para deixar claro algumas informações, foi usado groupby para sabermos a relação da quantidade de playlist(já limpas) com as categorias, o quê inclusive nos leva a observar e descobrir quais são as categorias do DataSet

In [16]:
playlists.groupby('category').count()

,playlists_limpas,timestamp
category,,
Alternating DJ,5053,5053
Ambient,306,306
Avant Garde,110,110
Blues,197,197
Break Up,1115,1115
Classical,73,73
Country,484,484
Cover,657,657
Dance/House,706,706


Tomamos então uma ação importante: 
Criaremos uma nova coluna chamada 'Playlist_Indicadas', e junto disso , adicionaremos o termo '__category__' seguido de um espaço e o nome da categoria pertecente a essa playlist. Isso será muito importante para algumas funções usando a bilioteca 'fasttext'.

In [17]:
playlists['Playlists_Indicadas'] = "__category__"+playlists['category']+" "+playlists['playlists_limpas']
playlists.head(3)

,category,playlists_limpas,timestamp,Playlists_Indicadas
0,Mixed Genre,PeterMurphy::MarleneDietrich'sFavouritePoem;Th...,2005-03-27 10:53:00,__category__Mixed Genre PeterMurphy::MarleneDi...
1,Theme,TheCure::EdgeOfTheDeepGreenSea;JeffBuckley::Ni...,2008-01-11 21:45:00,__category__Theme TheCure::EdgeOfTheDeepGreenS...
2,Theme,PeterGabriel::TheFeelingBegins;OfraHazaAndIggy...,2005-10-31 10:15:00,__category__Theme PeterGabriel::TheFeelingBegi...


Baixaremos  e importaremos uma importante biblioteca, que nos auxiliará na construção de arquivos vindo do nosso DataSet

In [18]:
!pip install scikit-learn

In [19]:
from sklearn.model_selection import train_test_split


Definiremos então tamanhos e proporções para os nossos arquivos de teste

In [20]:
train,test = train_test_split(playlists,test_size = 0.3)

In [21]:
display(train.shape)
display(test.shape)

(70940, 4)

(30403, 4)

Nesse momento, usando os seguintes passos, atribuiremos a um arquivo (esse passo pode ser executado criando um novo arquivo) linhas que contém, uma por uma, as informações de cada playlist

In [22]:
train.to_csv(r"C:\\Caio\\Owl\\Train and test\\train.csv",columns=['Playlists_Indicadas'],index=False,header=False)
test.to_csv(r"C:\\Caio\\Owl\\Train and test\\test.csv",columns=['Playlists_Indicadas'],index=False,header=False)

Treinaremos então um modelo , usando funções provinientes da biblioteca 'fasttext' e atribuindo um arquivo pra ela(o recém criado,train.csv).

In [23]:
modelo = fasttext.train_supervised(input=r"C:\\Caio\\Owl\\Train and test\\train.csv")

**LEMBRAR DE APAGAR ESSA CÉLULA**

Daqui em diante, os comandos deveriam realizar comando da biblioteca fasttext, porém geram erros no Kernell e "mata ele".
É necessário entender o porque disso acontece para prosseguir.
Se lerem isso, usei como base o vídeo de um indiano louco no youtube:"https://www.youtube.com/watch?v=ahYRKzYYCIw", copiei importantes passos dele. Ele basicamente está aplicando e ensinando como usar fasttext em um dataset.
Depois disso, teremos feito a maior parte do que o Flávio pediu, mas precisamos ainda de usar a Biblioteca Guenshim, porém , a prioridade agora é seguir no uso do fasttext.

LEMBREM DE MUDAR OS NOMES DAS PASTAS PARA RODAREM COM ÊXITO NO PCS DE VCS!!!

In [24]:
##modelo.test(r"C:\\Caio\\Owl\\Train and test\\test.csv")

In [25]:
##modelo.predict("TheAnimals")